In [1]:
import pandas as pd
import numpy as np
import os 
import re
import matplotlib.pyplot as plt
import scipy
# from scipy.stats.stats import pearsonr 
import os
from pathlib import Path
import math
from statsmodels.stats.multitest import multipletests
import statsmodels
from scipy.stats import ttest_rel, ttest_ind
from scipy import stats

In [2]:
normal =pd.read_csv('data/lusc-rsem-fpkm-tcga_paired.txt', sep="\t", header=0)
print(len(normal))
print(normal.columns)
print(normal.head(3))

19648
Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'TCGA-43-7657', 'TCGA-58-8386',
       'TCGA-22-5478', 'TCGA-22-5472', 'TCGA-43-5670', 'TCGA-60-2709',
       'TCGA-22-5489', 'TCGA-77-8007', 'TCGA-22-5471', 'TCGA-22-4609',
       'TCGA-22-5482', 'TCGA-56-8082', 'TCGA-22-5483', 'TCGA-56-8623',
       'TCGA-33-4587', 'TCGA-56-7579', 'TCGA-43-3394', 'TCGA-34-8454',
       'TCGA-77-7338', 'TCGA-43-6143', 'TCGA-43-6773', 'TCGA-51-4080',
       'TCGA-34-7107', 'TCGA-39-5040', 'TCGA-43-6771', 'TCGA-92-7340',
       'TCGA-77-7138', 'TCGA-77-7142', 'TCGA-56-7823', 'TCGA-22-5491',
       'TCGA-22-5481', 'TCGA-90-7767', 'TCGA-77-8008', 'TCGA-56-7730',
       'TCGA-33-6737', 'TCGA-56-7582', 'TCGA-56-7731', 'TCGA-77-7337',
       'TCGA-43-7658', 'TCGA-56-8201', 'TCGA-85-7710', 'TCGA-56-7580',
       'TCGA-43-6647', 'TCGA-90-6837', 'TCGA-56-8083', 'TCGA-51-4079',
       'TCGA-56-7222', 'TCGA-22-4593', 'TCGA-51-4081', 'TCGA-56-8309'],
      dtype='object')
  Hugo_Symbol  Entrez_Gene_Id  TCGA-43-7657  TC

In [3]:
cancer = pd.read_csv('data/lusc-rsem-fpkm-tcga-t_paired.txt', sep="\t", header=0)
print(len(cancer))
print(cancer.columns)
print(cancer.head(3))

19648
Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'TCGA-43-7657', 'TCGA-58-8386',
       'TCGA-22-5478', 'TCGA-22-5472', 'TCGA-43-5670', 'TCGA-60-2709',
       'TCGA-22-5489', 'TCGA-77-8007', 'TCGA-22-5471', 'TCGA-22-4609',
       'TCGA-22-5482', 'TCGA-56-8082', 'TCGA-22-5483', 'TCGA-56-8623',
       'TCGA-33-4587', 'TCGA-56-7579', 'TCGA-43-3394', 'TCGA-34-8454',
       'TCGA-77-7338', 'TCGA-43-6143', 'TCGA-43-6773', 'TCGA-51-4080',
       'TCGA-34-7107', 'TCGA-39-5040', 'TCGA-43-6771', 'TCGA-92-7340',
       'TCGA-77-7138', 'TCGA-77-7142', 'TCGA-56-7823', 'TCGA-22-5491',
       'TCGA-22-5481', 'TCGA-90-7767', 'TCGA-77-8008', 'TCGA-56-7730',
       'TCGA-33-6737', 'TCGA-56-7582', 'TCGA-56-7731', 'TCGA-77-7337',
       'TCGA-43-7658', 'TCGA-56-8201', 'TCGA-85-7710', 'TCGA-56-7580',
       'TCGA-43-6647', 'TCGA-90-6837', 'TCGA-56-8083', 'TCGA-51-4079',
       'TCGA-56-7222', 'TCGA-22-4593', 'TCGA-51-4081', 'TCGA-56-8309'],
      dtype='object')
  Hugo_Symbol  Entrez_Gene_Id  TCGA-43-7657  TC

In [4]:
def check(normal, cancer):
    if not (normal.columns == cancer.columns).all():
        print('Different Columns')
    if not (len(normal) == len(cancer)):
        print('Different length')
    else: 
        print('checked')
# print(normal.columns == cancer.columns)
# print(len(normal) == len(cancer))
check(normal, cancer)

checked


In [5]:
print((normal.columns == cancer.columns).all())

True


In [6]:
print(len(normal.columns))

52


In [7]:
normal_x = normal[((normal == 0).astype(int).sum(axis=1))<25]
cancer_x = cancer[((cancer == 0).astype(int).sum(axis=1))<25]
print(len(normal_x), len(cancer_x))

17626 17717


In [8]:
names = np.intersect1d(normal_x.Hugo_Symbol, cancer_x.Hugo_Symbol)

normal_x = normal_x[normal_x['Hugo_Symbol'].isin(names)]
cancer_x = cancer_x[cancer_x['Hugo_Symbol'].isin(names)]
normal_x.dropna()
cancer_x.dropna()
print(len(normal_x), len(cancer_x))

17275 17275


In [9]:
check(normal_x, cancer_x)

checked


In [10]:
np.array(normal_x.iloc[0][2:])

array([62.12, 130.6, 33.06, 35.5, 73.03, 60.39, 92.05, 66.65, 54.33,
       15.56, 55.49, 30.34, 14.45, 100.83, 131.51, 59.55, 87.03, 71.5,
       23.08, 48.87, 72.52, 115.97, 45.21, 74.58, 8.92, 27.44, 40.64,
       35.76, 59.13, 20.86, 37.05, 48.18, 51.71, 49.56, 72.01, 21.94,
       27.44, 35.0, 76.17, 28.86, 90.77, 59.55, 40.07, 22.92, 29.91,
       82.29, 4.7, 37.32, 43.63, 77.25], dtype=object)

In [11]:
dfs = [normal_x, cancer_x]
for df in dfs:
    df['paired'] = ''
    for i in range (len(df)):
        sample = np.array(df.iloc[i][2:-1])
        stat,pval = stats.shapiro(sample)
        if pval > 0.05 :
            df.at[i,'paired'] = 'ND'
        else: 
            df.at[i,'paired'] = 'NOT'
    df.head(3)
#     print('-----------')

In [12]:
# import inspect, re

# def varname(p):
#     for line in inspect.getframeinfo(inspect.currentframe().f_back)[3]:
#         m = re.search(r'\bvarname\s*\(\s*([A-Za-z_][A-Za-z0-9_]*)\s*\)', line)
#     if m:
#         return m.group(1)

# print (varname(normal_x))

In [13]:
counter = 0
x = ['normal', 'cancer']
for df in dfs:
    norm = len(df[df['paired'] == 'ND'])
    not_norm = len(df[df['paired'] == 'NOT'])
    print('dataframe {} has {} normally distrbuted genes and {} not normally distributed'.format( x[counter],norm, not_norm))
    counter +=1

dataframe normal has 6994 normally distrbuted genes and 10281 not normally distributed
dataframe cancer has 2028 normally distrbuted genes and 15247 not normally distributed


In [14]:
normal_norm = normal_x[normal_x['paired'] == 'ND']
normal_not = normal_x[normal_x['paired'] == 'NOT']
normal_norm.dropna()
normal_not.dropna()
print(len(normal_norm), len(normal_not))

cancer_norm = cancer_x[cancer_x['paired'] == 'ND']
cancer_not = cancer_x[cancer_x['paired'] == 'NOT']
cancer_norm.dropna()
cancer_not.dropna()
print(len(cancer_norm), len(cancer_not))

6994 10281
2028 15247


In [15]:
len(normal_norm.Hugo_Symbol.dropna())

6175

In [16]:
len(cancer_norm.Hugo_Symbol.dropna())

1772

In [17]:
normal_intersect = np.intersect1d(normal_norm.Hugo_Symbol.dropna(), cancer_norm.Hugo_Symbol.dropna())
not_intersect = np.intersect1d(normal_not.Hugo_Symbol.dropna(), cancer_not.Hugo_Symbol.dropna())
print(len(normal_intersect), len(not_intersect))

1124 8380


In [18]:
normal_norm = normal_norm[normal_norm['Hugo_Symbol'].isin(normal_intersect)]
cancer_norm = cancer_norm[cancer_norm['Hugo_Symbol'].isin(normal_intersect)]
print(len(normal_norm),len(cancer_norm))

1124 1124


In [19]:
normal_not = normal_not[normal_not['Hugo_Symbol'].isin(not_intersect)]
cancer_not = cancer_not[cancer_not['Hugo_Symbol'].isin(not_intersect)]
print(len(normal_not),len(cancer_not))

8380 8380


In [20]:
normal_norm.head(10)

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309,paired
50,C19orf44,84167.0,127.00,92.70,92.05,111.99,139.07,40.93,85.22,47.84,...,116.78,78.89,68.07,108.14,91.41,91.41,36.79,111.21,134.30,ND
75,TBC1D31,93594.0,70.51,78.34,63.00,77.79,80.57,23.42,59.13,88.26,...,91.41,36.01,52.45,49.21,35.00,54.72,50.27,64.34,76.17,ND
87,STX16,8675.0,575.03,896.64,607.87,543.96,854.13,591.22,521.76,674.59,...,988.12,559.28,536.45,718.08,669.92,866.07,497.00,921.88,780.44,ND
101,COL4A2,1284.0,7537.18,1259.69,2682.69,5831.91,4450.27,1769.57,4328.55,2047.00,...,2537.92,4010.71,17681.08,4358.66,6516.03,1242.34,3372.43,3168.41,4181.07,ND
114,ITGB6,3694.0,1104.13,1572.76,1104.13,1277.29,1143.10,1295.13,1322.37,2061.24,...,1059.11,1322.37,1295.13,836.53,1340.84,1216.75,1340.84,1088.92,738.29,ND
135,C14orf119,55017.0,1268.46,1757.34,1233.75,1369.04,1088.92,1883.54,1277.29,1909.85,...,1437.15,1378.57,1286.18,1081.39,1216.75,1407.55,1651.00,1467.37,1277.29,ND
172,ANKRD33B,651746.0,10.71,9.70,19.25,12.64,16.63,7.17,22.43,4.10,...,10.88,31.45,4.13,16.39,18.97,3.89,4.13,7.40,11.47,ND
177,MAD2L1,4085.0,74.06,105.89,35.76,92.05,85.82,49.91,67.59,221.86,...,122.64,31.90,66.65,41.22,34.75,26.86,63.89,47.84,108.90,ND
201,PARP11,57097.0,133.36,125.24,120.94,161.02,147.06,73.54,113.56,164.42,...,170.25,113.56,123.50,112.77,108.90,108.14,113.56,115.16,120.10,ND
218,TAF15,8148.0,2701.35,1617.00,2417.67,2701.35,2777.33,1304.15,2434.50,1498.22,...,2417.67,2468.49,2682.69,2384.37,2018.80,2417.67,2319.15,1806.78,2335.28,ND


In [21]:
cancer_norm.head(10)

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309,paired
50,C19orf44,84167.0,96.68,98.04,58.71,75.64,48.52,71.50,76.17,71.50,...,111.99,60.39,57.89,66.65,34.75,47.50,97.36,60.82,87.03,ND
75,TBC1D31,93594.0,187.71,226.54,125.24,99.43,158.79,127.00,152.28,202.66,...,138.10,88.26,143.01,129.69,124.37,289.02,129.69,130.60,88.26,ND
87,STX16,8675.0,808.00,1561.89,389.72,660.68,1001.93,884.29,540.19,974.50,...,629.35,733.19,753.83,1551.09,583.07,775.05,1447.15,1216.75,1096.50,ND
101,COL4A2,1284.0,2777.33,2520.38,1757.34,2434.50,688.78,4481.23,11664.82,3467.27,...,1159.07,7803.01,3280.18,4769.75,1605.83,3614.55,5995.90,3614.55,4123.49,ND
114,ITGB6,3694.0,236.21,830.75,132.44,728.11,934.76,426.57,1844.76,129.69,...,236.21,1388.16,620.67,277.20,486.75,842.36,297.17,94.01,743.43,ND
135,C14orf119,55017.0,947.83,1378.57,1498.22,1697.45,1583.71,1151.06,1167.14,1104.13,...,2018.80,1498.22,1331.57,1313.23,1540.37,1233.75,967.76,1709.26,1051.79,ND
172,ANKRD33B,651746.0,3.82,2.68,1.27,1.87,9.06,5.50,33.54,20.86,...,1.04,35.76,0.78,33.30,2.89,1.89,8.38,2.12,96.68,ND
177,MAD2L1,4085.0,400.71,473.41,423.61,289.02,665.29,518.15,429.54,389.72,...,1561.89,389.72,223.41,1216.75,656.11,896.64,1794.29,842.36,432.53,ND
201,PARP11,57097.0,75.11,83.45,183.82,157.68,76.71,142.01,379.04,93.35,...,135.24,133.36,96.68,211.31,81.71,132.44,100.13,170.25,171.45,ND
218,TAF15,8148.0,2400.97,1923.14,1651.00,1883.54,2645.74,2520.38,2178.83,2796.65,...,1844.76,2104.58,1605.83,3742.05,2090.03,2777.33,3280.18,2148.82,3081.75,ND


In [22]:
normal_not.head(10)

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309,paired
0,HIST3H2A,92815.0,62.12,130.60,33.06,35.50,73.03,60.39,92.05,66.65,...,59.55,40.07,22.92,29.91,82.29,4.70,37.32,43.63,77.25,NOT
1,LIN7B,64130.0,185.11,283.05,119.26,169.07,165.57,161.02,131.51,198.47,...,119.26,102.97,123.50,264.03,194.36,166.73,105.15,185.11,356.05,NOT
2,LXN,56925.0,909.17,819.30,412.00,743.43,1340.84,607.87,1709.26,1709.26,...,2400.97,543.96,2193.99,540.19,521.76,253.23,764.36,518.15,878.17,NOT
4,SCML1,6322.0,133.36,214.27,108.14,109.66,190.34,211.31,96.01,208.38,...,209.84,120.10,109.66,155.50,162.14,277.20,86.43,164.42,155.50,NOT
5,AC024592.12,0.0,25.91,32.13,17.38,13.72,21.01,28.86,23.59,34.02,...,20.56,32.13,28.86,11.30,18.84,33.78,23.93,29.70,20.26,NOT
6,GSDMD,79792.0,1733.13,2835.70,1508.65,1936.53,1819.35,2502.97,2090.03,2434.50,...,1950.00,2018.80,2004.85,2417.67,2209.26,1417.35,1733.13,2739.08,2835.70,NOT
7,AKR1C1,1645.0,1088.92,947.83,684.02,860.08,1096.50,2090.03,1369.04,1151.06,...,660.68,1051.79,743.43,961.07,1242.34,486.75,417.77,981.29,1733.13,NOT
9,CRISPLD2,83716.0,848.22,536.45,620.67,1216.75,1832.01,2287.20,693.58,836.53,...,579.04,2956.17,1233.75,1769.57,1781.89,511.00,2255.70,2193.99,457.25,NOT
12,GLI1,2735.0,75.64,32.82,15.34,99.43,51.71,34.02,13.72,68.07,...,45.85,43.32,98.73,28.65,56.68,20.41,127.00,40.07,34.75,NOT
13,STK17B,9262.0,595.34,1175.27,1104.13,595.34,518.15,1023.00,575.03,830.75,...,414.87,547.75,567.10,444.72,775.05,902.89,848.22,638.15,616.37,NOT


In [23]:
cancer_not.head(10)

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309,paired
0,HIST3H2A,92815.0,336.79,500.46,703.28,287.01,486.75,70.51,145.02,14.03,...,420.68,109.66,106.63,1233.75,172.65,303.44,228.13,251.48,23.59,NOT
1,LIN7B,64130.0,105.15,212.78,102.25,212.78,172.65,244.57,105.89,152.28,...,135.24,151.22,395.18,295.11,120.94,114.36,96.68,277.20,153.34,NOT
2,LXN,56925.0,848.22,236.21,271.48,759.08,61.25,620.67,329.84,599.49,...,204.07,438.59,503.95,3039.30,607.87,106.63,507.46,255.00,346.29,NOT
4,SCML1,6322.0,84.63,74.58,67.12,57.89,102.97,132.44,66.65,57.08,...,119.26,87.65,53.57,232.94,67.12,64.80,90.14,198.47,154.42,NOT
5,AC024592.12,0.0,17.13,25.91,16.88,63.45,27.84,23.08,46.50,27.05,...,32.13,32.59,21.63,5.19,28.65,14.45,27.05,16.63,29.48,NOT
6,GSDMD,79792.0,1551.09,1427.22,1674.06,1685.71,3124.78,2133.97,2451.44,2240.11,...,1135.20,1832.01,1208.34,1883.54,988.12,941.27,909.17,1135.20,2004.85,NOT
7,AKR1C1,1645.0,9945.68,723.08,1023.00,1242.34,136.19,40621.74,660.68,84.04,...,1175.27,1143.10,656.11,334.46,52135.28,2133.97,934.76,37379.55,5403.70,NOT
9,CRISPLD2,83716.0,162.14,297.17,518.15,220.32,185.11,1015.93,1059.11,591.22,...,262.20,2319.15,1023.00,287.01,438.59,182.55,902.89,507.46,830.75,NOT
12,GLI1,2735.0,84.63,246.28,21.63,31.90,14.03,169.07,24.11,10.47,...,56.28,27.84,92.05,92.05,102.25,11.38,50.98,633.73,28.24,NOT
13,STK17B,9262.0,145.02,373.81,320.80,371.22,205.50,336.79,633.73,463.65,...,267.73,470.14,242.88,180.02,486.75,551.56,239.52,223.41,620.67,NOT
